In [1]:
import pandas as pd
import dbconfig as dbConf

## 총합 양수 음수 갯수 구하는 함수

In [375]:
def pstv_ngtv_cnt(sum_num):
    # MDM DB 연결
    zeroDb = dbConf.DbConfig("zero")
    zeroDb.opendb()
    sql = (f"""
            select sum_num
                 , sum(case when diff > 0 then 1 end) as pstv
                 , sum(case when diff < 0 then 1 end) as ngtv
              from (
            select a.*
                 , ifnull((select (b.n1 + b.n2 + b.n3 + b.n4 + b.n5 + b.n6) - sum_num from temp b where b.seq = (a.seq + 1)),0) as diff
              from (
                    select a.seq
                         , a.n1, a.n2, a.n3, a.n4, a.n5, a.n6
                         , (a.n1 + a.n2 + a.n3 + a.n4 + a.n5 + a.n6) as sum_num
                      from temp a
                   ) a
             where 1=1
               and sum_num = {sum_num}
                   ) b 
             group by sum_num
           """
           )

    # 수집 대상 DB정보 및 테이블, 쿼리 조회
    #print(sql)
    source = zeroDb.select(sql) 
    zeroDb.closedb()

    return pd.DataFrame(source)

## 패턴으로 결과값 뽑기

In [344]:
def Patten_GetNumber(max_val, f1, f2, f3, f4):
    # MDM DB 연결
    zeroDb = dbConf.DbConfig("zero")
    zeroDb.opendb()
    sql = (f"""
             select sum(case when row_num = 1 then num end) as n1
                  , sum(case when row_num = 2 then num end) as n2
                  , sum(case when row_num = 3 then num end) as n3
                  , sum(case when row_num = 4 then num end) as n4
                  , sum(case when row_num = 5 then num end) as n5
                  , sum(case when row_num = 6 then num end) as n6
                  , '' as bin
                  , sum(case when row_num = 1 then seq end) as f1
                  , sum(case when row_num = 2 then seq end) as f2
                  , sum(case when row_num = 3 then seq end) as f3
                  , sum(case when row_num = 4 then seq end) as f4
                  , sum(case when row_num = 5 then seq end) as f5
                  , sum(case when row_num = 6 then seq end) as f6
               from (
             select num
                  , @rownum:=@rownum+1 as row_num
                  , seq
               from (
                    select seq, num   
                      from (select 3 as seq, num
                              from in_list 
                             where seq = {max_val} 
                               and val = 3 
                             order by rand() 
                             limit {f1}
                           ) as t3
                    union all
                    select seq, num   
                      from (select 5 as seq, num
                              from in_list 
                             where seq = {max_val}
                               and val = 5 
                             order by rand() 
                             limit {f2}
                           ) as t5 
                    union all
                    select seq, num   
                      from (select 10 as seq, num
                              from in_list 
                             where seq = {max_val}
                               and val = 10
                             order by rand() 
                             limit {f3}
                           ) as t10
                    union all
                    select seq, num   
                    from (select 30 as seq, num
                            from in_list 
                           where seq = {max_val}
                             and val = 30 
                           order by rand() 
                           limit {f4}
                         ) as t30
                     order by num 
                 ) as a
                 , (SELECT @rownum:=0) TMP
                 ) as b 
             ;
           """
           )

    # 수집 대상 DB정보 및 테이블, 쿼리 조회
    #print(sql)
    source = zeroDb.select(sql) 
    zeroDb.closedb()

    return pd.DataFrame(source)

## sql 다이렉트로 날릴수 있는 함수

In [343]:
def sql_direct(sql_list):
    # MDM DB 연결
    zeroDb = dbConf.DbConfig("zero")
    zeroDb.opendb()
    sql = sql_list

    # 수집 대상 DB정보 및 테이블, 쿼리 조회
    #print(sql)
    source = zeroDb.select(sql) 
    zeroDb.closedb()

    return pd.DataFrame(source)

## 패턴 얻기

In [372]:
def GetPatten():
    # MDM DB 연결
    zeroDb = dbConf.DbConfig("zero")
    zeroDb.opendb()
    #print("접속완료")

    sql = ("""
           select stan
                , 3cnt
                , 5cnt
                , 10cnt
                , 30cnt
             from in_list_2 a
            order by 1 desc
            limit 10
           """
           )

    # 수집 대상 DB정보 및 테이블, 쿼리 조회
    source = zeroDb.select(sql) 
    zeroDb.closedb()

    return pd.DataFrame(source)

In [385]:
# 패턴 뽑기
df = GetPatten()

stan    = list(range(0,1))
cnt3    = list(range(0,1))
cnt5    = list(range(0,1))
cnt10   = list(range(0,1))
cnt30   = list(range(0,1))

for df_stan, df_cnt3, df_cnt5, df_cnt10, df_cnt30 in zip(df['stan'], df['3cnt'], df['5cnt'], df['10cnt'], df['30cnt']):
    stan.append(int(df_stan))
    cnt3.append(int(df_cnt3))
    cnt5.append(int(df_cnt5))
    cnt10.append(int(df_cnt10))
    cnt30.append(int(df_cnt30))

In [401]:
df

,stan,3cnt,5cnt,10cnt,30cnt
0,1016,3,1,2,0
1,1015,3,0,1,2
2,1014,1,0,4,1
3,1013,3,0,1,2
4,1012,3,1,1,1
5,1011,1,1,2,2
6,1010,2,0,3,1
7,1009,4,0,1,1
8,1008,2,1,0,3
9,1007,1,2,1,2


## 전체 결과 

In [400]:
# 전체 내용
sql = """
select a.*
     , b.pstv
     , b.ngtv
     , ''
     , c.3cnt
     , c.5cnt
     , c.10cnt
     , c.30cnt
  from (
select a.seq
	 , a.n1, a.n2, a.n3, a.n4, a.n5, a.n6
	 , (a.n1 + a.n2 + a.n3 + a.n4 + a.n5 + a.n6) as sum_num
	 , ifnull((select (b.n1 + b.n2 + b.n3 + b.n4 + b.n5 + b.n6) - (a.n1 + a.n2 + a.n3 + a.n4 + a.n5 + a.n6) 
				 from temp b 
				where b.seq = (a.seq + 1)),0) as diff
  from temp a 
       ) a left outer join (select sum_num
								 , ifnull(sum(case when diff > 0 then 1 end),0) as pstv
								 , ifnull(sum(case when diff < 0 then 1 end),0) as ngtv
							  from (
							select (a.n1 + a.n2 + a.n3 + a.n4 + a.n5 + a.n6) as sum_num
								 , ifnull((select (b.n1 + b.n2 + b.n3 + b.n4 + b.n5 + b.n6) - (a.n1 + a.n2 + a.n3 + a.n4 + a.n5 + a.n6) 
											 from temp b 
											where b.seq = (a.seq + 1)),0) as diff
							  from temp a
								   ) a
							 group by sum_num
						   ) b
					    on a.sum_num = b.sum_num 
		   left outer join in_list_2 c
                        on a.seq = c.stan 
 order by 1 desc
 limit 10
       
"""

sql_all_data = sql_direct(sql)

In [402]:
sql_all_data

,seq,n1,n2,n3,n4,n5,n6,sum_num,diff,pstv,ngtv,,3cnt,5cnt,10cnt,30cnt
0,1017,12,18,22,23,30,34,139,0,4,7,,None,None,None,None
1,1016,15,26,28,34,41,42,186,-47,0,2,,3,1,2,0
2,1015,14,23,31,33,37,40,178,8,2,5,,3,0,1,2
3,1014,3,11,14,18,26,27,99,79,5,1,,1,0,4,1
4,1013,21,22,26,34,36,41,180,-81,0,3,,3,0,1,2
5,1012,5,11,18,20,35,45,134,46,7,6,,3,1,1,1
6,1011,1,9,12,26,35,38,121,13,10,2,,1,1,2,2
7,1010,9,12,15,25,34,36,131,-10,9,6,,2,0,3,1
8,1009,15,23,29,34,40,44,185,-54,0,4,,4,0,1,1
9,1008,9,11,30,31,41,44,166,19,1,6,,2,1,0,3


-------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------

## 데이터 뽑기

In [334]:
# 최신 패턴으로 결과 뽑기
data = Patten_GetNumber(stan[1],0,0,0,0)

f1 = 3
f2 = 1
f3 = 1
f4 = 1

for i in range(0, 5):
    data = data.append(Patten_GetNumber(stan[1]+1, f1, f2, f3, f4), ignore_index = True)
    data['num_sum'] = data['n1'] + data['n2'] + data['n3'] + data['n4'] + data['n5'] + data['n6'] 
    
data

,n1,n2,n3,n4,n5,n6,bin,f1,f2,f3,f4,f5,f6,num_sum
0,None,None,None,None,None,None,,None,None,None,None,None,None,NaN
1,4,18,21,26,42,45,,30,3,5,3,3,10,156
2,5,15,22,27,32,33,,10,3,3,5,30,3,134
3,13,15,21,38,40,42,,30,3,5,10,3,3,169
4,15,21,24,28,35,41,,3,5,30,3,10,3,164
5,2,3,14,28,38,41,,30,5,3,3,10,3,126
